In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np  # numpy 임포트 필요

# 1. 데이터 불러오기
try:
    data = pd.read_csv("/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_2/Data/CarPrice.csv")
except FileNotFoundError:
    print("CarPrice.csv 파일을 찾을 수 없습니다. 파일 경로를 확인하세요.")
    # 파일이 없으면 더 이상 진행하지 않음


# 2. 데이터 전처리
# 종속 변수(y) 및 독립 변수(X) 분리
y = data['price']

# X에서 불필요한 변수 제거 (car_ID: 단순 식별자, CarName: 카디널리티가 너무 높아 모델에 부정적 영향, price: 종속 변수)
X = data.drop(['price', 'car_ID', 'CarName'], axis=1)


#Alogirthm
# [1: 'object' 타입의 숫자형 변수 수동 변환]
# 'doornumber'와 'cylindernumber'는 순서가 있는 수치형 변수이므로 직접 매핑.
num_map = {
    'two': 2, 'three': 3, 'four': 4, 'five': 5,
    'six': 6, 'eight': 8, 'twelve': 12
}
# .map()을 사용하면 매핑되지 않은 값(예: NaN)이 발생할 수 있으나,
# 뒤따르는 fillna()에서 처리되므로 안전합니다.
X['doornumber'] = X['doornumber'].map(num_map)
X['cylindernumber'] = X['cylindernumber'].map(num_map)


# 3. 범주형 변수 원-핫 인코딩
# 이제 X에 남은 'object' 타입은 실제 범주형 변수들임 (예: fueltype, carbody)
X_processed = pd.get_dummies(X, drop_first=True)


# [2: '?'와 같은 문자열로 인해 'object'가 된 숫자 열 처리]
# X_processed의 모든 열을 강제로 숫자형으로 변환합니다.
# 변환할 수 없는 값(예: '?')은 'coerce' 옵션으로 NaN (Not a Number)이 됩니다.
# 이 단계가 핵심임.
for col in X_processed.columns:
    X_processed[col] = pd.to_numeric(X_processed[col], errors='coerce')


# [3: NaN 값 처리]
# statsmodels는 NaN 값을 허용하지 않는다.
# pd.to_numeric(errors='coerce')로 인해 생긴 NaN 값이나,
# 수동 매핑(num_map)에서 변환되지 못한 NaN 값들을 각 열의 '평균값'으로 대체하여 대입함.
X_processed = X_processed.fillna(X_processed.mean())

# 이제 X_processed는 모든 열이 숫자형(float)이고 NaN 값이 없음.


# 4. 절편(Intercept) 추가
# statsmodels는 sklearn과 달리 수동으로 절편(상수항)을 추가해야 함
X_with_const = sm.add_constant(X_processed)

# [수정 4: 최종 데이터 타입 확인]
# OLS에 입력하기 전, 모든 데이터 타입을 float으로 명확하게 지정함.
X_with_const = X_with_const.astype(float)


# 5. 모델 적합 (Baseline Multiple Linear Regression)
# OLS (Ordinary Least Squares) 모델 학습
model = sm.OLS(y, X_with_const)
results = model.fit()


# 6. 결과 보고 및 해석
print("--- [ 1.(a) 문제 보고서 (수정됨) ] ---")
print("\n[보고 항목 1: 절편 및 표준화 여부]")
print("1. 절편(Intercept) 포함 여부: 예, 모델에 절편을 포함했습니다.")
print("2. 예측 변수 표준화 여부: 아니오, 기본 다중 선형 회귀 모델에서는 표준화를 적용하지 않았습니다.")

print("\n[보고 항목 2: 정규화 모델에서 표준화가 중요한 이유]")
print(
    "릿지(Ridge)와 라쏘(Lasso) 같은 정규화 회귀는 계수(beta)의 크기 자체에 페널티를 부과합니다.\n"
    "만약 예측 변수들이 서로 다른 스케일(예: 'horsepower' (50~200) vs 'boreratio' (2~4))을 갖는다면,\n"
    "스케일이 큰 변수의 계수가 스케일이 작은 변수의 계수보다 불공평하게 더 큰 페널티를 받게 됩니다.\n"
    "이는 모델이 변수의 중요도가 아닌 스케일에 의존하여 계수를 축소하게 만듭니다.\n"
    "따라서, 모든 변수가 페널티 항에 동등하게 기여하도록 만들기 위해 (Z-score 표준화 등으로)\n"
    "데이터의 스케일을 통일(평균 0, 분산 1)하는 표준화 전처리가 필수적입니다."
)

print("\n[보고 항목 3: 계수, 유의미한 계수, 훈련 R^2]")

r_squared = results.rsquared
all_coefficients = results.params
p_values = results.pvalues

# 유의수준 (p<0.05) 기준
significant_coefficients = all_coefficients[p_values < 0.05]

print(f"\n1. 훈련 데이터 R^2 (Training R-squared): {r_squared:.4f}")

print("\n2. 추정된 모든 계수 (절편 포함):")
print(all_coefficients.to_string())

print("\n3. 통계적으로 유의미한 계수 (p < 0.05):")
if significant_coefficients.empty:
    print("유의미한 계수가 없습니다.")
else:
    print(significant_coefficients.to_string())

print("\n--- [ 보고서 종료 ] ---")

--- [ 1.(a) 문제 보고서 (수정됨) ] ---

[보고 항목 1: 절편 및 표준화 여부]
1. 절편(Intercept) 포함 여부: 예, 모델에 절편을 포함했습니다.
2. 예측 변수 표준화 여부: 아니오, 기본 다중 선형 회귀 모델에서는 표준화를 적용하지 않았습니다.

[보고 항목 2: 정규화 모델에서 표준화가 중요한 이유]
릿지(Ridge)와 라쏘(Lasso) 같은 정규화 회귀는 계수(beta)의 크기 자체에 페널티를 부과합니다.
만약 예측 변수들이 서로 다른 스케일(예: 'horsepower' (50~200) vs 'boreratio' (2~4))을 갖는다면,
스케일이 큰 변수의 계수가 스케일이 작은 변수의 계수보다 불공평하게 더 큰 페널티를 받게 됩니다.
이는 모델이 변수의 중요도가 아닌 스케일에 의존하여 계수를 축소하게 만듭니다.
따라서, 모든 변수가 페널티 항에 동등하게 기여하도록 만들기 위해 (Z-score 표준화 등으로)
데이터의 스케일을 통일(평균 0, 분산 1)하는 표준화 전처리가 필수적입니다.

[보고 항목 3: 계수, 유의미한 계수, 훈련 R^2]

1. 훈련 데이터 R^2 (Training R-squared): 0.9249

3. 통계적으로 유의미한 계수 (p < 0.05):
const                 -28085.137699
carwidth                 643.437554
carheight                272.508363
curbweight                 3.719531
enginesize               143.857745
boreratio              -3612.571522
stroke                 -4131.864103
compressionratio       -1573.675798
peakrpm                    2.532420
fueltype_gas          -24523.375835
carbody_hard